In [1]:
# Import dependencies
import os

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

2025-06-16 09:49:41.112777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750060181.191951    3518 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750060181.215410    3518 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750060181.401980    3518 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750060181.401999    3518 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750060181.402000    3518 computation_placer.cc:177] computation placer alr


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 16 intra-op, 9 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1750060184.679317    3518 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1750060184.680082    3518 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping, filter_no_finding_class
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights, oversample_minority_classes
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "DenseNetV5"
model_name      = f"{set_name}_1"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

train_ds = filter_no_finding_class(train_ds)
val_ds   = filter_no_finding_class(val_ds)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 104166 samples [00:35, 2927.91 samples/s] 
Counting samples: 15431 samples [00:08, 1716.94 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryFocalCrossentropy(gamma=2.0, from_logits=False, label_smoothing=0.05)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max", save_weights_only=False)

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max", save_weights_only=False)

2025-06-16 09:51:46 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV5_1/train_metrics.csv
2025-06-16 09:51:46 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV5_1/val_metrics.csv
2025-06-16 09:51:46 - INFO - Cleaned training metrics file, kept 37398 records before epoch 47
2025-06-16 09:51:46 - INFO - Cleaned validation metrics file, kept 46 records before epoch 47
2025-06-16 09:51:46 - INFO - Found 37398 existing training records
2025-06-16 09:51:46 - INFO - Found 46 existing validation records


In [5]:
# Model Training
model          = build_densenet121(num_classes, use_se=True, ratio=16)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-06-16 09:51:47 - INFO - Resuming training from checkpoint: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV5_1/checkpoints/cp-0046.keras
2025-06-16 09:51:50 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV5_1/train_metrics.csv
2025-06-16 09:51:50 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV5_1/val_metrics.csv
2025-06-16 09:51:50 - INFO - Cleaned training metrics file, kept 37398 records before epoch 47
2025-06-16 09:51:50 - INFO - Cleaned validation metrics file, kept 46 records before epoch 47
2025-06-16 09:51:50 - INFO - Found 37398 existing training records
2025-06-16 09:51:50 - INFO - Found 46 existing validation records
2025-06-16 09:51:50 - INFO - 
=== Training Started ===

2025-06-16 09:51:50 - INFO - Batch Size: 128
2025-06-16 09:51:50 - INFO - Optimizer: Adam
2025-06-16 09:51:50 - INFO - 

2025-06-16 09:51:5

Epoch 47/100


I0000 00:00:1750060334.936176    4023 service.cc:152] XLA service 0x7038881d4eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750060334.936230    4023 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1750060342.411689    4023 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750060409.386636    4023 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.9435 - auc: 0.9288 - f1_score: 0.5105 - loss: 0.0298 - precision: 0.7414 - recall: 0.4266  

2025-06-16 09:56:55 - INFO - 
=== Epoch 47 Summary ===
2025-06-16 09:56:55 - INFO - Time: 305.38s
2025-06-16 09:56:55 - INFO - Training   - accuracy: 0.9442 - auc: 0.9307 - f1_score: 0.5162 - loss: 0.0295 - precision: 0.7423 - recall: 0.4329 - learning_rate: 0.0000
2025-06-16 09:56:55 - INFO - Validation - accuracy: 0.9352 - auc: 0.7592 - f1_score: 0.2066 - loss: 0.0629 - precision: 0.3345 - recall: 0.1713
2025-06-16 09:56:55 - INFO - ==================================================



...

Epoch 83/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.9496 - auc: 0.9461 - f1_score: 0.5883 - loss: 0.0265 - precision: 0.7744 - recall: 0.5030  

2025-06-16 11:59:17 - INFO - 
=== Epoch 83 Summary ===
2025-06-16 11:59:17 - INFO - Time: 204.16s
2025-06-16 11:59:17 - INFO - Training   - accuracy: 0.9498 - auc: 0.9467 - f1_score: 0.5883 - loss: 0.0264 - precision: 0.7735 - recall: 0.5037 - learning_rate: 0.0000
2025-06-16 11:59:17 - INFO - Validation - accuracy: 0.9334 - auc: 0.7521 - f1_score: 0.2270 - loss: 0.0675 - precision: 0.3270 - recall: 0.1913
2025-06-16 11:59:17 - INFO - ==================================================



813/813 ━━━━━━━━━━━━━━━━━━━━ 204s 251ms/step - accuracy: 0.9496 - auc: 0.9461 - f1_score: 0.5883 - loss: 0.0265 - precision: 0.7744 - recall: 0.5030 - val_accuracy: 0.9334 - val_auc: 0.7521 - val_f1_score: 0.2270 - val_loss: 0.0675 - val_precision: 0.3270 - val_recall: 0.1913 - learning_rate: 1.0000e-06


2025-06-16 11:59:17 - INFO - 
=== Training Completed! ===

2025-06-16 11:59:17 - INFO - Final Metrics: accuracy: 0.9498 - auc: 0.9467 - f1_score: 0.5883 - loss: 0.0264 - precision: 0.7735 - recall: 0.5037 - val_accuracy: 0.9334 - val_auc: 0.7521 - val_f1_score: 0.2270 - val_loss: 0.0675 - val_precision: 0.3270 - val_recall: 0.1913



Epoch 83: early stopping
Restoring model weights from the end of the best epoch: 68.
